In [ ]:
# Import the necessary libraries
%matplotlib inline
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
import pickle

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score as acs_score

from src.data.data_fetcher import get_raw_data
from src.features.preprocess_data import get_preprocessed_test_data, fetch_preprocessed_data
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Prepare data
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
x_test_whole = get_preprocessed_test_data()

x_whole = pd.concat([X_train_obs_combined, X_val_obs_combined])
y_whole = pd.concat([y_train_obs_combined, y_val_obs_combined])
x_whole.reset_index(drop=True, inplace=True)
y_whole.reset_index(drop=True, inplace=True)

x_whole_obs = pd.concat([X_train_obs_combined, X_val_obs_combined])
y_whole_obs = pd.concat([y_train_obs_combined, y_val_obs_combined])

x_whole_est = pd.concat([X_train_est_combined, X_val_est_combined])
y_whole_est = pd.concat([y_train_est_combined, y_val_est_combined])

x_whole.head()

In [ ]:
x_whole["pv_measurement"] = y_whole
df_shuffled = x_whole.sample(frac=1, random_state=42).reset_index(drop=True)
x_whole_a = df_shuffled[df_shuffled['location_a'] == 1]
x_whole_b = df_shuffled[df_shuffled['location_b'] == 1]
x_whole_c = df_shuffled[df_shuffled['location_c'] == 1]

y_whole_a = x_whole_a["pv_measurement"]
x_whole_a = x_whole_a.drop("pv_measurement", axis = 1)
x_whole_a = x_whole_a.drop('location_a', axis = 1)
x_whole_a = x_whole_a.drop('location_b', axis = 1)
x_whole_a = x_whole_a.drop('location_c', axis = 1)

y_whole_b = x_whole_b["pv_measurement"]
x_whole_b = x_whole_b.drop("pv_measurement", axis = 1)
x_whole_b = x_whole_b.drop('location_a', axis = 1)
x_whole_b = x_whole_b.drop('location_b', axis = 1)
x_whole_b = x_whole_b.drop('location_c', axis = 1)

y_whole_c = x_whole_c["pv_measurement"]
x_whole_c = x_whole_c.drop("pv_measurement", axis = 1)
x_whole_c = x_whole_c.drop('location_a', axis = 1)
x_whole_c = x_whole_c.drop('location_b', axis = 1)
x_whole_c = x_whole_c.drop('location_c', axis = 1)

In [ ]:
import optuna

def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'depth': trial.suggest_int('depth', 4, 10),
        'iterations': trial.suggest_int('iterations', 100, 2000),  # Optimizing the number of iterations
        'eval_metric': 'MAE',
        'random_seed': 42,
        'verbose': 200,
        'loss_function': 'MAE',
        # Add more parameters here if you want
    }
    
    fold_mae = []
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    for train_index, test_index in kf.split(y_whole_a):
        X_train_fold, X_val_fold = x_whole_a.iloc[train_index], x_whole_a.iloc[test_index]
        y_train_fold, y_val_fold = y_whole_a.iloc[train_index], y_whole_a.iloc[test_index]
        
        model = CatBoostRegressor(**param)
        model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], early_stopping_rounds=100)
        
        y_pred_fold = model.predict(X_val_fold)
        fold_mae.append(mean_absolute_error(y_val_fold, y_pred_fold))
    
    return np.mean(fold_mae)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # You can increase n_trials to try more combinations

# Best hyperparameters
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

# Retrain with best parameters
best_params = study.best_trial.params
best_model_a = CatBoostRegressor(**best_params)
best_model_a.fit(x_whole_a, y_whole_a)

In [ ]:
import optuna

def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'depth': trial.suggest_int('depth', 4, 10),
        'iterations': trial.suggest_int('iterations', 100, 2000),  # Optimizing the number of iterations
        'eval_metric': 'MAE',
        'random_seed': 42,
        'verbose': 200,
        'loss_function': 'MAE',
        # Add more parameters here if you want
    }
    
    fold_mae = []
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    for train_index, test_index in kf.split(x_whole_b):
        X_train_fold, X_val_fold = x_whole_b.iloc[train_index], x_whole_b.iloc[test_index]
        y_train_fold, y_val_fold = y_whole_b.iloc[train_index], y_whole_b.iloc[test_index]
        
        model = CatBoostRegressor(**param)
        model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], early_stopping_rounds=100)
        
        y_pred_fold = model.predict(X_val_fold)
        fold_mae.append(mean_absolute_error(y_val_fold, y_pred_fold))
    
    return np.mean(fold_mae)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # You can increase n_trials to try more combinations

# Best hyperparameters
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

# Retrain with best parameters
best_params = study.best_trial.params
best_model_b = CatBoostRegressor(**best_params)
best_model_b.fit(x_whole_b, y_whole_b)


In [ ]:
import optuna

def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 10),
        'depth': trial.suggest_int('depth', 4, 10),
        'iterations': trial.suggest_int('iterations', 100, 2000),  # Optimizing the number of iterations
        'eval_metric': 'MAE',
        'random_seed': 42,
        'verbose': 200,
        'loss_function': 'MAE',
        # Add more parameters here if you want
    }
    
    fold_mae = []
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    for train_index, test_index in kf.split(x_whole_c):
        X_train_fold, X_val_fold = x_whole_c.iloc[train_index], x_whole_c.iloc[test_index]
        y_train_fold, y_val_fold = y_whole_c.iloc[train_index], y_whole_c.iloc[test_index]
        
        model = CatBoostRegressor(**param)
        model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], early_stopping_rounds=100)
        
        y_pred_fold = model.predict(X_val_fold)
        fold_mae.append(mean_absolute_error(y_val_fold, y_pred_fold))
    
    return np.mean(fold_mae)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # You can increase n_trials to try more combinations

# Best hyperparameters
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

# Retrain with best parameters
best_params = study.best_trial.params
best_model_c = CatBoostRegressor(**best_params)
best_model_c.fit(x_whole_c, y_whole_c)


In [ ]:
x_whole_a = x_test_whole[x_test_whole['location_a'] == 1]
x_whole_b = x_test_whole[x_test_whole['location_b'] == 1]
x_whole_c = x_test_whole[x_test_whole['location_c'] == 1]

x_whole_a = x_whole_a.drop('location_a', axis = 1)
x_whole_a = x_whole_a.drop('location_b', axis = 1)
x_whole_a = x_whole_a.drop('location_c', axis = 1)

x_whole_b = x_whole_b.drop('location_a', axis = 1)
x_whole_b = x_whole_b.drop('location_b', axis = 1)
x_whole_b = x_whole_b.drop('location_c', axis = 1)

x_whole_c = x_whole_c.drop('location_a', axis = 1)
x_whole_c = x_whole_c.drop('location_b', axis = 1)
x_whole_c = x_whole_c.drop('location_c', axis = 1)

In [ ]:
def multi_predict(x_values :pd.DataFrame, models) -> pd.DataFrame:
    """
    Function for predicting on multiple models and averaging the results
    """
    results = models[0].predict(x_values)
    for model in models[1:]:
        model: xgb.XGBRegressor
        prediction = model.predict(x_values)
        results += prediction
    
    results = results / len(models)

    return results


In [ ]:
y_pred_b = best_model_a.predict(x_whole_a)
y_pred_c = best_model_b.predict(x_whole_b)
y_pred_a = best_model_c.predict(x_whole_c)
y_pred = pd.concat([y_pred_a, y_pred_b, y_pred_c])
y_pred = y_pred.reset_index(drop=True)
# Save the model
from src.models.saving import save_predictions
from src.features.postprocess_data import postprocess_data

y_pred = postprocess_data(x_test_whole, pd.DataFrame(y_pred))
save_predictions(y_pred, 'catboost optuna 3 modeller 1')